# Guidance for training a model with your own data

## 1. Import the necessary packages

In [1]:
import argparse
import os
import random

import numpy as np
import pandas as pd
import torch

from softs.exp.exp_custom import Exp_Custom

## 2. Define the hyperparameters

In [2]:
# fix seed for reproducibility
fix_seed = 2021
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)
torch.set_num_threads(6)

# basic config
config = {
    # dataset settings
    "root_path": "./dataset/ETT-small/",
    "data_path": "ETTm1.csv",
    "data": "ETTm1",
    "features": "M",
    "freq": "h",
    "seq_len": 96,
    "pred_len": 96,
    # model settings
    "model": "SOFTS",
    "checkpoints": "./checkpoints/",
    "d_model": 128,
    "d_core": 64,
    "d_ff": 128,
    "e_layers": 2,
    "learning_rate": 0.0003,
    "lradj": "cosine",
    "train_epochs": 50,
    "patience": 3,
    "batch_size": 16,
    "dropout": 0.0,
    "activation": "gelu",
    "use_norm": True,
    # system settings
    "num_workers": 0,
    "use_gpu": True,
    "gpu": "0",
    "save_model": True,
    "loss_func": "huber",
}

parser = argparse.ArgumentParser(description='SOFTS')
args = parser.parse_args([])
args.__dict__.update(config)
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

print('Args in experiment:')
print(args)

Args in experiment:
Namespace(root_path='./dataset/ETT-small/', data_path='ETTm1.csv', data='ETTm1', features='M', freq='h', seq_len=96, pred_len=96, model='SOFTS', checkpoints='./checkpoints/', d_model=128, d_core=64, d_ff=128, e_layers=2, learning_rate=0.0003, lradj='cosine', train_epochs=50, patience=3, batch_size=16, dropout=0.0, activation='gelu', use_norm=True, num_workers=0, use_gpu=False, gpu='0', save_model=True, loss_func='huber')


## 3. Prepare the dataset
Organize your data in the following format:
- The dataset should be a csv file.
- If there is a time feature, the first column contains timestamps in the format 'YYYY-MM-DD HH:MM:SS'. If there's no time feature, the dataset starts directly with the features.
- If the parameter `features` is 'M', the following columns are both the features and the targets. If `features` is 'MS', the following columns are the features, and the last column is the target.

In [3]:
# load data
data = pd.read_csv(os.path.join(args.root_path, args.data_path))
print(data.head())

# split data
train_data = data.iloc[: 12 * 30 * 24 * 4]
vali_data = data.iloc[12 * 30 * 24 * 4 - args.seq_len: 12 * 30 * 24 * 4 + 4 * 30 * 24 * 4]
test_data = data.iloc[12 * 30 * 24 * 4 + 4 * 30 * 24 * 4 - args.seq_len: 12 * 30 * 24 * 4 + 8 * 30 * 24 * 4]

# optional: scale data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
if 'date' in train_data.columns:
    scaler.fit(train_data.iloc[:, 1:])
    train_data.iloc[:, 1:] = scaler.transform(train_data.iloc[:, 1:])
    vali_data.iloc[:, 1:] = scaler.transform(vali_data.iloc[:, 1:])
    test_data.iloc[:, 1:] = scaler.transform(test_data.iloc[:, 1:])
else:
    scaler.fit(train_data.iloc[:, :])
    train_data.iloc[:, :] = scaler.transform(train_data.iloc[:, :])
    vali_data.iloc[:, :] = scaler.transform(vali_data.iloc[:, :])
    test_data.iloc[:, :] = scaler.transform(test_data.iloc[:, :])

                  date   HUFL   HULL   MUFL   MULL   LUFL   LULL         OT
0  2016-07-01 00:00:00  5.827  2.009  1.599  0.462  4.203  1.340  30.531000
1  2016-07-01 00:15:00  5.760  2.076  1.492  0.426  4.264  1.401  30.459999
2  2016-07-01 00:30:00  5.760  1.942  1.492  0.391  4.234  1.310  30.038000
3  2016-07-01 00:45:00  5.760  1.942  1.492  0.426  4.234  1.310  27.013000
4  2016-07-01 01:00:00  5.693  2.076  1.492  0.426  4.142  1.371  27.787001


## 4. Train and Evaluate the model


In [4]:
Exp = Exp_Custom(args)
setting = f'{args.data}_{args.model}_{args.seq_len}_{args.pred_len}'
print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
Exp.train(setting=setting, train_data=train_data, vali_data=vali_data, test_data=test_data)
print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
Exp.test(setting=setting, test_data=test_data)

Use CPU
>>>>>>>start training : ETTm1_SOFTS_96_96>>>>>>>>>>>>>>>>>>>>>>>>>>
	iters: 100, epoch: 1 | loss: 0.1774055
	speed: 0.0318s/iter; left time: 3410.6829s
	iters: 200, epoch: 1 | loss: 0.1016285
	speed: 0.0211s/iter; left time: 2259.9480s
	iters: 300, epoch: 1 | loss: 0.1712743
	speed: 0.0249s/iter; left time: 2672.9192s
	iters: 400, epoch: 1 | loss: 0.1048555
	speed: 0.0295s/iter; left time: 3154.4056s
	iters: 500, epoch: 1 | loss: 0.1215826
	speed: 0.0280s/iter; left time: 2995.3636s
	iters: 600, epoch: 1 | loss: 0.1622979
	speed: 0.0256s/iter; left time: 2734.6055s
	iters: 700, epoch: 1 | loss: 0.1366529
	speed: 0.0242s/iter; left time: 2585.4898s
	iters: 800, epoch: 1 | loss: 0.1319874
	speed: 0.0249s/iter; left time: 2652.5566s
	iters: 900, epoch: 1 | loss: 0.1074991
	speed: 0.0232s/iter; left time: 2474.1516s
	iters: 1000, epoch: 1 | loss: 0.1267864
	speed: 0.0238s/iter; left time: 2536.1783s
	iters: 1100, epoch: 1 | loss: 0.1280163
	speed: 0.0245s/iter; left time: 2607.8372

(0.5345862657740525, 0.35867902898031984, 0.13361184783230987)

## 5. Get predictions by the model

In [6]:
# get predictions
predictions = Exp.predict(setting=setting, pred_data=test_data)
print(predictions.shape)

loading model from ./checkpoints/ETTm1_SOFTS_96_96/checkpoint.pth
(11521, 96, 7)
